In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install neo4j
!pip install tqdm
!pip install wikidata
!pip install pywikibot
!pip install pandas
!pip install SPARQLWrapper

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pywikibot from https://files.pythonhosted.org/packages/37/b1/9db28152805b17948eeb5efab1e091aeeb53ee939efe09cb7f8e2f821521/pywikibot-8.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for mwparserfromhell>=0.5.2 from https://files.pythonhosted.org/packages/9c/72

## World Knowledge Graph Turtle to Neo4j Knowledge Graph

The main goal is to re-create a knowledge graph of the world by using WorldKG

### Loading Neo4j

In [4]:
from kgce.neo4j.handler import Neo4jWrapper
from kgce.schema.semantic.neo4jclasses import Neo4jNode,Neo4jGraph,Neo4jRelation
from tqdm import tqdm

In [5]:
neoWrapper = Neo4jWrapper('bolt://localhost:7687','neo4j','12345678')

In [6]:
# neograph = Neo4jNode()

### Creating nodes out of planet.ttl

In [5]:
def List2Node(block):
    dict_params = {}
    dict_params['id'] = block[0].replace(" ;","").replace(" .","").replace("    ","").replace("\n","").split(" ")[0]
    for i in range(1, len(block)):
        if block[i] != "\n":
            split_pair = block[i].replace(" ;","").replace(" .","").replace("    ","").replace("\n","").split(" ")
            property_name = split_pair[0]
            value = " ".join(split_pair[1:])
            dict_params[property_name] = value
    name = block[1].replace(" ;","").replace(" .","").replace("    ","").replace("\n","").replace('"','').split(" ")[1:]
    name = " ".join(name)
    # Returns node
    return Neo4jNode(name=name,
                        class_name=block[0].replace(" ;","").replace("\n","").split(" ")[-1], 
                        dict_param=dict_params)

In [6]:
# Results
node_dictionary = {}
node_without_wikidata = []
# Temporary
last_object = []
prefix = False
node = None
# Create a node per line of this document
with open("graphplanet_osm.ttl") as file:
    for line in file:
        last_object += [line]
        # If end of block
        if line == "\n":
            # if it is not the prefix
            if prefix:
                node = List2Node(last_object)
                # if has wikidata
                if "wkgs:wikidata" in node.dict_param:
                    node_dictionary[node.dict_param["wkgs:wikidata"]] = node
                else:
                    node_without_wikidata.append(node)
            # Prefixes
            else:
                prefix = True
            # Reset
            last_object = []

In [7]:
# Overall nodes Statistics Nodes with wikidata link : 275933
# Nodes without wikidata link : 1032489
## Nodes with wikidata link
print("Nodes with wikidata link : " + str(len(node_dictionary.keys())))
## Nodes without wikidata link
print("Nodes without wikidata link : " + str(len(node_without_wikidata)))

Nodes with wikidata link : 720930
Nodes without wikidata link : 587492


In [7]:
def Nodes2Turtle(nodes):
    with open("graphplanet_osm.ttl", 'w') as fd:
        fd.write("")
    with open("graphplanet_osm.ttl", '+a') as fd:
        fd.write("@prefix geo: <http://www.opengis.net/ont/geosparql#> .\n@prefix wdp: <https://www.wikidata.org/prop/> .\n@prefix osmn: <https://www.openstreetmap.org/node/> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix sf: <http://www.opengis.net/ont/sf#> .\n@prefix wd: <http://www.wikidata.org/wiki/> .\n@prefix wkg: <http://www.worldkg.org/resource/> .\n@prefix wkgs: <http://www.worldkg.org/schema/> .\n\n")
        for node in nodes:
            fd.write(node.dict_param["id"] + " a " + node.class_name  + " ;\n")
            keylist = list(node.dict_param.keys())
            for paramnb in range(0,len(keylist)):
                paramid = keylist[paramnb]
                if paramid != "id":
                    fd.write("    " + paramid + " " + node.dict_param[paramid] + (" ;\n" if paramnb+1 < len(keylist) else " .\n\n"))

In [27]:
Nodes2Turtle(node_without_wikidata + list(node_dictionary.values()))

### Enrichment of nodes without wikidata link :

In [8]:
from pywikibot.data import api
import pywikibot

def getItems(site, itemtitle):
    params = { 'action' :'wbsearchentities' , 'format' : 'json' , 'language' : 'en', 'type' : 'item', 'search': itemtitle}
    request = api.Request(site=site,**params)
    return request.submit()

# Login to wikidata
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

In [12]:
for node in tqdm(node_without_wikidata):
    wikidataEntries = getItems(site, node.name)
    if len(wikidataEntries['search']) > 0:
        qid = "wd:" + wikidataEntries['search'][0]['id']
        if qid not in node_dictionary.keys():
            node_dictionary[qid] = node

  0%|          | 0/1032489 [00:00<?, ?it/s]WARNING: /tmp/ipykernel_5910/895355822.py:6: DeprecationWarning: Instead of using kwargs from Request.__init__, parameters for the request to the API should be added via the "parameters" parameter.
  request = api.Request(site=site,**params)

100%|██████████| 1032489/1032489 [51:34:27<00:00,  5.56it/s]  


In [19]:
for key in tqdm(node_dictionary.keys()):
	node = node_dictionary[key]
	if 'wkgs:wikidata' not in node.dict_param:
		node.dict_param['wkgs:wikidata'] = key
	if node in node_without_wikidata:
		node_without_wikidata.remove(node)

100%|██████████| 720930/720930 [4:08:08<00:00, 48.42it/s]  


In [9]:
import requests

API_ENDPOINT = "https://www.wikidata.org/w/api.php"
query = "Q43463"
params = {
'action': 'wbgetentities',
'format': 'json',
'languages': 'de',
'ids': query,
'props': 'labels|descriptions|claims'
}
r = requests.get(API_ENDPOINT, params = params)
print(r.json())


{'entities': {'Q43463': {'type': 'item', 'id': 'Q43463', 'labels': {'de': {'language': 'de', 'value': 'Fortaleza'}}, 'descriptions': {'de': {'language': 'de', 'value': 'Stadt in Brasilien'}}, 'claims': {'P856': [{'mainsnak': {'snaktype': 'value', 'property': 'P856', 'hash': '810c3a8e5cb6dd07558f2e27e879f42307450c36', 'datavalue': {'value': 'http://www.fortaleza.ce.gov.br/', 'type': 'string'}, 'datatype': 'url'}, 'type': 'statement', 'qualifiers': {'P407': [{'snaktype': 'value', 'property': 'P407', 'hash': 'd3341820d45f6bb2097e8460656f7cd8fea3cf2a', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 5146, 'id': 'Q5146'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}]}, 'qualifiers-order': ['P407'], 'id': 'Q43463$4f806e8e-4434-16cd-a634-4c085e79297d', 'rank': 'normal'}], 'P190': [{'mainsnak': {'snaktype': 'value', 'property': 'P190', 'hash': '8e5df7326bd8ccd141078c7391fd496706449c16', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 201516, 'id': 'Q2015

In [12]:
r.json()['entities']["Q43463"]["claims"].keys()

dict_keys(['P856', 'P190', 'P17', 'P31', 'P373', 'P41', 'P94', 'P18', 'P242', 'P281', 'P421', 'P473', 'P625', 'P131', 'P948', 'P910', 'P982', 'P646', 'P935', 'P1464', 'P1566', 'P227', 'P1740', 'P1792', 'P1376', 'P1465', 'P1082', 'P47', 'P571', 'P2044', 'P150', 'P2046', 'P1081', 'P138', 'P214', 'P1997', 'P1585', 'P244', 'P2163', 'P3417', 'P1225', 'P3219', 'P1549', 'P1417', 'P402', 'P1313', 'P194', 'P5573', 'P6555', 'P1296', 'P6766', 'P1245', 'P7867', 'P443', 'P1343', 'P7859', 'P1889', 'P8744', 'P973', 'P9100', 'P2716', 'P898', 'P206', 'P7471', 'P8168', 'P85', 'P463', 'P7305', 'P163', 'P237', 'P417', 'P2184', 'P2633', 'P361', 'P8189', 'P4342', 'P8253', 'P3984', 'P8313', 'P1831', 'P10565'])

In [14]:
import sys
import pandas as pd
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON

class WikiDataQueryResults:
    """
    A class that can be used to query data from Wikidata using SPARQL and return the results as a Pandas DataFrame or a list
    of values for a specific key.
    """
    def __init__(self, query: str):
        """
        Initializes the WikiDataQueryResults object with a SPARQL query string.
        :param query: A SPARQL query string.
        """
        self.user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        self.endpoint_url = "https://query.wikidata.org/sparql"
        self.sparql = SPARQLWrapper(self.endpoint_url, agent=self.user_agent)
        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

    def __transform2dicts(self, results: List[Dict]) -> List[Dict]:
        """
        Helper function to transform SPARQL query results into a list of dictionaries.
        :param results: A list of query results returned by SPARQLWrapper.
        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        new_results = []
        for result in results:
            new_result = {}
            for key in result:
                new_result[key] = result[key]['value']
            new_results.append(new_result)
        return new_results

    def _load(self) -> List[Dict]:
        """
        Helper function that loads the data from Wikidata using the SPARQLWrapper library, and transforms the results into
        a list of dictionaries.
        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        results = self.sparql.queryAndConvert()['results']['bindings']
        results = self.__transform2dicts(results)
        return results

    def load_as_dataframe(self) -> pd.DataFrame:
        """
        Executes the SPARQL query and returns the results as a Pandas DataFrame.
        :return: A Pandas DataFrame representing the query results.
        """
        results = self._load()
        return pd.DataFrame.from_dict(results)


In [11]:
query = '''
SELECT ?ps_ ?p {
  VALUES (?company) {(wd:Q43463)}

  ?company ?p ?statement .
  ?statement ?ps ?ps_ .

  ?wd wikibase:claim ?p.
  ?wd wikibase:statementProperty ?ps.
  
  FILTER (strstarts(str(?ps_), 'http://www.wikidata.org/entity/'))

  OPTIONAL {
  ?statement ?pq ?pq_ .
  ?wdpq wikibase:qualifier ?pq .
  }
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
} ORDER BY ?wd ?statement ?ps_
'''
data_extracter = WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
print(df.head())

                                   p                                       ps_
0  http://www.wikidata.org/prop/P131     http://www.wikidata.org/entity/Q40123
1  http://www.wikidata.org/prop/P138   http://www.wikidata.org/entity/Q2914125
2  http://www.wikidata.org/prop/P150  http://www.wikidata.org/entity/Q10369218
3  http://www.wikidata.org/prop/P150  http://www.wikidata.org/entity/Q20059453
4  http://www.wikidata.org/prop/P150  http://www.wikidata.org/entity/Q10369215


In [15]:
def GetPropertiesFrom(qid):
	query = '''
	SELECT ?ps_ ?p {
	  VALUES (?company) {(wd:'''+qid+''')}

	  ?company ?p ?statement .
	  ?statement ?ps ?ps_ .

	  ?wd wikibase:claim ?p.
	  ?wd wikibase:statementProperty ?ps.
	  
	  FILTER (strstarts(str(?ps_), 'http://www.wikidata.org/entity/'))

	  OPTIONAL {
	  ?statement ?pq ?pq_ .
	  ?wdpq wikibase:qualifier ?pq .
	  }
	  
	  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
	} ORDER BY ?wd ?statement ?ps_
	'''
	data_extracter = WikiDataQueryResults(query)
	df = data_extracter.load_as_dataframe()
	return df

In [18]:
dict_node_relation = {}  
for key in tqdm(node_dictionary.keys()): 
	try:
		df = GetPropertiesFrom(key.replace("wd:",""))
		list_properties = []
		for index in range(df.shape[0]):
			prop = "wdp:" + df['p'][index].split("/")[-1]
			qvalue = "wd:" + df['ps_'][index].split("/")[-1]
			if qvalue in node_dictionary.keys(): 
				list_properties.append((prop,qvalue))
		dict_node_relation[key] = list_properties
	except:
		print("Problem with " + key)

  7%|▋         | 49524/720930 [2:54:01<34:29:21,  5.41it/s] 

Problem with wd:Q5827588


  9%|▉         | 67079/720930 [3:54:26<32:24:24,  5.60it/s] 

Problem with "?"


 89%|████████▊ | 638133/720930 [37:43:06<4:31:11,  5.09it/s]   

Problem with wd:Q31857426


100%|██████████| 720930/720930 [42:41:17<00:00,  4.69it/s]   


In [30]:
def NodesAndEdges2Turtle(nodes, edges_dict):
    with open("newgraphplanet_osm.ttl", 'w') as fd:
        fd.write("")
    with open("newgraphplanet_osm.ttl", '+a') as fd:
        fd.write("@prefix geo: <http://www.opengis.net/ont/geosparql#> .\n@prefix wdp: <https://www.wikidata.org/prop/> .\n@prefix osmn: <https://www.openstreetmap.org/node/> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix sf: <http://www.opengis.net/ont/sf#> .\n@prefix wd: <http://www.wikidata.org/wiki/> .\n@prefix wkg: <http://www.worldkg.org/resource/> .\n@prefix wkgs: <http://www.worldkg.org/schema/> .\n\n")
        for node in nodes:
            fd.write(node.dict_param["id"] + " a " + node.class_name  + " ;\n")
            if "wkgs:wikidata" in node.dict_param.keys() and node.dict_param["wkgs:wikidata"] in edges_dict.keys():
                edgekeylist = edges_dict[node.dict_param["wkgs:wikidata"]]
                for i in range(0,len(edgekeylist)):
                    fd.write("    " + edgekeylist[i][0] + " " + edgekeylist[i][1] + " ;\n")
            keylist = list(node.dict_param.keys())
            for paramnb in range(0,len(keylist)):
                paramid = keylist[paramnb]
                if paramid != "id":
                    fd.write("    " + paramid + " " + node.dict_param[paramid] + (" ;\n" if paramnb+1 < len(keylist) else " .\n\n"))

In [27]:
node_dictionary[list(node_dictionary.keys())[1]].dict_param["id"]

'wkg:1001313724'

In [31]:
NodesAndEdges2Turtle(node_without_wikidata + list(node_dictionary.values() ),dict_node_relation) 

### Read enriched data and use our pkg model

In [13]:
def EnrichedList2Node(block):
    # First we get all nodes
    dict_params = {}
    dict_params['id'] = block[0].replace(" ;","").replace(" .","").replace("    ","").replace("\n","").split(" ")[0]
    name = ""
    for i in range(1, len(block)):
        if block[i] != "\n":
            split_pair = block[i].replace(" ;","").replace(" .","").replace("    ","").replace("\n","").split(" ")
            property_name = split_pair[0]
            value = " ".join(split_pair[1:])
            if property_name.split(":")[0] == "wdp":
                continue
            elif property_name == "rdfs:label":
                name = value
            dict_params[property_name] = value
    # Returns node
    return Neo4jNode(name=name,
    class_name=block[0].replace(" ;","").replace("\n","").split(" ")[-1], 
    dict_param=dict_params)

In [14]:
# Results
node_dictionary = {}
node_without_wikidata = []
# Temporary
last_object = []
prefix = False
node = None
# Create a node per line of this document
with open("newgraphplanet_osm.ttl") as file:
    for line in file:
        last_object += [line]
        # If end of block
        if line == "\n":
            # if it is not the prefix
            if prefix:
                node = EnrichedList2Node(last_object)
                # if has wikidata
                if "wkgs:wikidata" in node.dict_param:
                    node_dictionary[node.dict_param["wkgs:wikidata"]] = node
                else:
                    node_without_wikidata.append(node)
            # Prefixes
            else:
                prefix = True
            # Reset
            last_object = []

In [25]:
def EnrichedList2Edges(dict_nodes, neograph, block):
    # First we get all nodes
    wikidata_id = "None"
    edge_list = []
    for i in range(1, len(block)):
        if block[i] != "\n":
            split_pair = block[i].replace(" ;","").replace(" .","").replace("    ","").replace("\n","").split(" ")
            property_name = split_pair[0]
            value = " ".join(split_pair[1:])
            if property_name.split(":")[0] == "wdp":
                edge_list.append((property_name,value))
            elif property_name == "wkgs:wikidata":
                wikidata_id = value
    if wikidata_id == "None":
        return
    stnode = dict_nodes[wikidata_id]
    for edge in edge_list:
        endnode = dict_nodes[edge[1]]
        neograph.add_unidirectional_edge(stnode,endnode,Neo4jRelation(name=edge[0],
                                                                      class_name=edge[0]))

In [26]:
neograph = Neo4jGraph("WorldKG",node_without_wikidata + list(node_dictionary.values()))

In [27]:
last_object=[]
# Create edges per block of document
with open("newgraphplanet_osm.ttl") as file:
    for line in tqdm(file):
        last_object += [line]
        # If end of block
        if line == "\n":
            # if it is not the prefix
            if prefix:
                EnrichedList2Edges(node_dictionary, neograph, last_object)
            # Prefixes
            else:
                prefix = True
            # Reset
            last_object = []

12484177it [21:01:44, 164.91it/s]


In [31]:
neograph.edges_dict

{(<kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf81532970>,
  <kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf80efb0d0>): [<kgce.schema.semantic.neo4jclasses.Neo4jRelation at 0x7faf3a120850>],
 (<kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf81532970>,
  <kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf80d95f70>): [<kgce.schema.semantic.neo4jclasses.Neo4jRelation at 0x7faf3a120f40>],
 (<kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf81532970>,
  <kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf5808a160>): [<kgce.schema.semantic.neo4jclasses.Neo4jRelation at 0x7faf33b46e50>],
 (<kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf815329d0>,
  <kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf80d657f0>): [<kgce.schema.semantic.neo4jclasses.Neo4jRelation at 0x7faf3a120f10>],
 (<kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf81532a30>,
  <kgce.schema.semantic.neo4jclasses.Neo4jNode at 0x7faf80daeaf0>): [<kgce.schema.semantic.neo4jclasses.Neo4j

### Creating csv tables

#### Creating nodes

In [105]:
import pandas as pd

def retrieve_columns(neograph):
    columns = set()
    for node in neograph.nodes:
        for param in node.dict_param:
            columns.add(param.replace(":","_"))
    return columns

In [229]:
columns = list(retrieve_columns(neograph))
columns[columns.index("id")] = "id:ID"
columns.insert(0,"Name")
columns.insert(0,":LABEL")

In [230]:
columns_dict = dict(zip(columns, [[] for i in range(len(columns))]))

In [161]:
position_to_skip = []
ids = []
print("Check for unique nodes")
for node in tqdm(neograph.nodes):
    if node.dict_param["id"] in ids:
        position_to_skip.append(ids.index(node.dict_param["id"]))
    ids.append(node.dict_param["id"])

Check for unique nodes


100%|██████████████████████████████| 1308422/1308422 [12:14:46<00:00, 29.68it/s]


In [183]:
node_id_to_skip = []
for i in position_to_skip:
    node_id_to_skip.append(neograph.nodes[i].dict_param["id"])

In [186]:
import pickle

with open('node_id_to_skip.txt', 'wb') as fp:
    pickle.dump(node_id_to_skip, fp)

In [231]:
import numpy as np

def neograph_nodes2csv(neograph, columns_dict, position_to_skip):
    print("Extract dict")
    count = -1
    for node in tqdm(neograph.nodes):
        count += 1
        if count in position_to_skip:
            continue
        columns_dict[":LABEL"].append(node.class_name)
        columns_dict["id:ID"].append(node.dict_param['id'])
        columns_dict["Name"].append(node.name)
        for column in columns_dict.keys():
            if column == "id:ID" or column == ":LABEL" or column == "Name":
                continue
            if column.replace("_",":") in node.dict_param.keys():
                columns_dict[column].append(node.dict_param[column.replace("_",":")])
            else:
                columns_dict[column].append(np.nan)

In [232]:
neograph_nodes2csv(neograph, columns_dict,position_to_skip)

Extract dict


100%|███████████████████████████████| 1308422/1308422 [06:39<00:00, 3273.62it/s]


In [233]:
df = pd.DataFrame(data=columns_dict)

In [234]:
df.to_csv("worldkg_nodes.csv",index=False)

### Edges

In [210]:
import numpy as np

def neograph_edges2csv(neograph,node_id_to_skip):
    columns_dict = {":START_ID":[], ":END_ID":[], ":TYPE":[]}
    for stnode, endnode in tqdm(neograph.edges_dict.keys()):
        for edge in neograph.edges_dict[(stnode, endnode)]:
            columns_dict[":START_ID"].append(stnode.dict_param['id'])
            columns_dict[":END_ID"].append(endnode.dict_param['id'])
            if stnode.dict_param["id"] in node_id_to_skip:
                print("st node " + stnode.dict_param["id"])
            if endnode.dict_param["id"] in node_id_to_skip:
                print("end node " + endnode.dict_param["id"])
            columns_dict[":TYPE"].append(edge.class_name)
    return columns_dict

In [ ]:
df = pd.DataFrame(data=edge_columns_dict)
df.to_csv("worldkg_edges.csv",index=False)

[autoreload of _frozen_importlib failed: Traceback (most recent call last):
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/usr/lib/python3.8/importlib/__init__.py", line 166, in reload
    spec = module.__spec__ = _bootstrap._find_spec(name, pkgpath, target)
AttributeError: module 'importlib._bootstrap' has no attribute '_find_spec'
]
[autoreload of _frozen_importlib_external failed: Traceback (most recent call last):
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/usr/l

[autoreload of signal failed: Traceback (most recent call last):
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 361, in update_class
    update_instances(old, new)
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 319, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment: 'Signals' object layout differs from 'Signals'
]
[autoreload of inspect failed: Traceback (most recent call last):
  File "/home/gabriel/.

Unexpected exception formatting exception. Falling back to standard exception


[autoreload of asyncio failed: Traceback (most recent call last):
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/usr/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 848, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/lib/python3.8/asyncio/__init__.py", line 26, in <module>
    __all__ = (base_events.__all__ +
NameError: name 'base_events' is not defined
]
[autoreload of ctypes._endian failed: Traceback (most recent call last):
  File "/home/gabriel/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.p

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/gabriel/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/gabriel/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 736, in start
    self.io_loop.start()
  File "/home/gabriel/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 1845, in _run_once
    if handle._cancelled:
TypeError: descriptor 